In [ ]:
import requests
import json
import pandas as pd
import urllib.parse
import time
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
ARTS_TABLES = ['List-OfficerBeneficiaryMailing', 'Affiliate']
BENFI_TABLES = ['Individual']

TODAY_DATE = (date.today()).strftime("%m/%d/%Y")

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.BENFI_API_KEY = 'e78349ac-c900-4089-83a0-66a86ae4e185'
        self.BENFI_APP_ID = '6124ef7965dc077ddc4fc0ce'
        
        self.ARTS_API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.ARTS_APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.ARTS_GET_HEADERS = {'X-Knack-REST-API-KEY':self.ARTS_API_KEY,'X-Knack-Application-Id':self.ARTS_APP_ID}
        self.ARTS_POST_HEADERS = {'X-Knack-REST-API-KEY':self.ARTS_API_KEY,'X-Knack-Application-Id':self.ARTS_APP_ID,'content-type':'application/json'}
        
        self.BENFI_GET_HEADERS = {'X-Knack-REST-API-KEY':self.BENFI_API_KEY,'X-Knack-Application-Id':self.BENFI_APP_ID}
        self.BENFI_POST_HEADERS = {'X-Knack-REST-API-KEY':self.BENFI_API_KEY,'X-Knack-Application-Id':self.BENFI_APP_ID,'content-type':'application/json'}
        
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/'

        # INTERNAL #
        self.ARTS_APP_DICT = {}
        self.BENFI_APP_DICT = {}
        self.INV_BENFI_DICT = {}

        res = requests.get(url=self.LOADER_URL+self.ARTS_APP_ID)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in ARTS_TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower().replace(' ',''):item['key']})
                self.ARTS_APP_DICT.update({name:{'obj_id':key,'fields':fields}})
                
        res = requests.get(url=self.LOADER_URL+self.BENFI_APP_ID)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            inv_fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in BENFI_TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower().replace(' ',''):item['key']})
                    inv_fields.update({item['key']:item['name'].lower().replace(' ','')})
                self.BENFI_APP_DICT.update({name:{'obj_id':key,'fields':fields}})
                self.INV_BENFI_DICT.update({name:{'obj_id':key,'fields':inv_fields}})
    
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.BENFI_APP_DICT[object_name]['fields'][field_name]
        object_id = self.BENFI_APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + object_id + "/records?filters=" + filter_for_url + "&rows_per_page=1000"
        
        r = requests.get(url = request_url, headers = self.BENFI_GET_HEADERS)

        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def find_record(self, knack_object, number):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['uid']

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': number}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]
        else:
            return ''

    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.ARTS_APP_DICT[connection_name]['obj_id']
        knack_field_id = self.ARTS_APP_DICT[connection_name]['fields'][field]

        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        
        r = requests.get(url = request_url, headers = self.ARTS_GET_HEADERS)
        res_json_dict = json.loads(json.dumps(r.json()))

        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''

client = KnackAFT()
adict = client.ARTS_APP_DICT
bdict = client.BENFI_APP_DICT


print('Getting new records from Beneficiary Knack...')
records = client.find_matches('Individual', 'mdate', TODAY_DATE)
output_df = []
if records:
    for record in records:
        record_dict = {}
        data_payload = {}
        data_out = {}
        for k,v in record.items():
            if k != 'id':
                if 'raw' in k:
                    record_dict.update({client.INV_BENFI_DICT['Individual']['fields'][k.strip('_raw')]+'_raw':v})
                else:
                    record_dict.update({client.INV_BENFI_DICT['Individual']['fields'][k]:v})
        print(record_dict['individualname_raw'])
        
        aff_conn = client.get_connection('Affiliate','affiliatenumber',record_dict['affiliatenumber_raw'][0]['identifier'].zfill(5)) if record_dict['affiliatenumber_raw'] else ''
        currentemail = record_dict["currentemail_raw"]["email"] if record_dict["currentemail"] else ''
        firstname = record_dict["individualname_raw"]["first"] if 'first' in record_dict["individualname_raw"] else ''
        lastname = record_dict["individualname_raw"]["last"] if 'last' in record_dict["individualname_raw"] else ''
        street = record_dict["currentaddress_raw"]["street"] if 'street' in record_dict["currentaddress_raw"] else ''
        street2 = record_dict["currentaddress_raw"]["street2"] if 'street2' in record_dict["currentaddress_raw"] else ''
        city = record_dict["currentaddress_raw"]["city"] if 'city' in record_dict["currentaddress_raw"] else ''
        state = record_dict["currentaddress_raw"]["state"] if 'state' in record_dict["currentaddress_raw"] else ''
        zipcode = record_dict["currentaddress_raw"]["zip"] if 'zip' in record_dict["currentaddress_raw"] else ''
        
        data_payload.update({"link":record_dict['link_raw']['url']})
        data_payload.update({"currentofficerrole":record_dict['currentofficerrole']})
        data_payload.update({"termstart":record_dict['termstart']})
        data_payload.update({"termend":record_dict['termend']})
        data_payload.update({"individualguid":record_dict['individualguid']})
        
        data_payload.update({"filedate": TODAY_DATE})
        data_payload.update({"affiliate":aff_conn})
        
        for k,v in data_payload.items():
            data_out.update({adict['List-OfficerBeneficiaryMailing']['fields'][k]:v})
        data_out.update({"field_728": {"email": currentemail}})
        data_out.update({"field_732": {
                "street": street,
                "street2": street2,
                "city": city,
                "state": state,
                "zip": zipcode
            }})
        data_out.update({"field_731": {"first": firstname,"last": lastname}})
        print(f"Creating new record in ARTS for individual guid: {record_dict['individualguid']}...")
        request_url = "https://api.knack.aft.org/v1/objects/" + adict['List-OfficerBeneficiaryMailing']['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.ARTS_POST_HEADERS, data = json.dumps(data_out))
        print(r)
        time.sleep(1)
        print('---------------------------------------')